# BioExplorer - Astrocytes
![](../bioexplorer_neurons_banner.png)

In [ ]:
import os
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm
import libsonata
import numpy as np
import morphio
import glob
import h5py

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'astrocytes'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
data_folder = os.getenv('ASTROCYTES_DATA_FOLDER')
print(data_folder)

## Circuit

In [ ]:
circuit_config = os.path.join(data_folder, 'build', 'ngv_config.json')
circuit = libsonata.CircuitConfig.from_file(circuit_config)
population_guid = 1

In [ ]:
json = json.loads(circuit.expanded_json)
json.keys()

### Connectivities

In [ ]:
import h5py
connectivities = json['connectivities']
gliovascular = connectivities['gliovascular']
endfeet_areas = gliovascular['endfeet_areas']

endfeet_areas_filename = os.path.join(data_folder, 'build', endfeet_areas)
print(endfeet_areas_filename)
endfeets = h5py.File(endfeet_areas_filename)

#### End-feet areas

In [ ]:
gliovascular_filename = os.path.join(data_folder, 'build', 'sonata', 'edges', 'gliovascular.h5')
gliovascular = libsonata.EdgeStorage(gliovascular_filename)

In [ ]:
import numpy as np
gliovascular_population = gliovascular.open_population('gliovascular')
ids = np.arange(0, gliovascular_population.size)
astrocyte_section_ids = gliovascular_population.get_attribute('astrocyte_section_id', list(ids))
endfoot_ids = gliovascular_population.get_attribute('endfoot_id', list(ids))

endfoot_section_ids = dict()
for i in range(len(astrocyte_section_ids)):
    endfoot_section_ids[endfoot_ids[i]] = int(astrocyte_section_ids[i])

In [ ]:
import numpy as np

with Session(engine) as session:
    session.execute('DELETE FROM %s.end_foot' % db_schema)
    session.commit()

objects = endfeets['objects']
for obj in objects:
    guid = int(obj.replace('endfoot_', ''))
    vertices = objects[obj]['points']
    v = list()
    for vertex in vertices:
        v.append(vertex[0])
        v.append(vertex[1])
        v.append(vertex[2])
    vertices_buffer = np.array(v, dtype=np.float32).tobytes()
    indices = objects[obj]['triangles']
    i = list()
    for index in indices:
        i.append(index[0])
        i.append(index[1])
        i.append(index[2])
    indices_buffer = np.array(i, dtype=np.uint32).tobytes()
    astrocyte_guid = gliovascular_population.target_nodes([guid])[0]
    astrocyte_section_guid = int(astrocyte_section_ids[guid])

    with Session(engine) as session:
        session.execute(
            'INSERT INTO %s.end_foot VALUES (:guid, :astrocyte_guid, :astrocyte_section_guid, :vertices, :indices)' % db_schema,
            {
                'guid': guid,
                'astrocyte_guid': int(astrocyte_guid),
                'astrocyte_section_guid': astrocyte_section_guid,
                'vertices': vertices_buffer,
                'indices': indices_buffer,
            }
        )
        session.commit()

### Components

In [ ]:
components = json['components']
print(components)

In [ ]:
for component in components:
    with Session(engine) as session:
        session.execute(
            'DELETE FROM %s.configuration WHERE guid=:guid' % db_schema,
            {
                'guid': component
            }
        )
        session.commit()

    print('Populating components')
    with Session(engine) as session:
        session.execute(
            'INSERT INTO %s.configuration VALUES (:guid, :value)' % db_schema,
            {
                'guid': component,
                'value': components[component],
            }
        )
        session.commit()    

### Nodes

In [ ]:
nodes = circuit.node_population('astrocytes')
ids = np.arange(0, nodes.size)

model_templates = nodes.get_attribute('model_template', list(ids))
model_types = nodes.get_attribute('model_type', list(ids))
morphologies = nodes.get_attribute('morphology', list(ids))
mtypes = nodes.get_attribute('mtype', list(ids))
radii = nodes.get_attribute('radius', list(ids))
xs = nodes.get_attribute('x', list(ids))
ys = nodes.get_attribute('y', list(ids))
zs = nodes.get_attribute('z', list(ids))

In [ ]:
# Nodes
print('Deleting existing nodes')
with Session(engine) as session:
    session.execute(
        'DELETE FROM %s.node WHERE population_guid=:population_guid' % db_schema,
        {
            'population_guid': population_guid
        }
    )
    session.commit()

print('Populating nodes')
with Session(engine) as session:
    for i in tqdm(range(nodes.size)):
        session.execute(
            'INSERT INTO %s.node VALUES (:guid, :population_guid, :x, :y, :z, :radius, :model_template, :model_type, :morphology, :mtype)' % db_schema,
            {
                'guid': i,
                'population_guid': population_guid,
                'x': xs[i],
                'y': ys[i],
                'z': zs[i],
                'radius': radii[i],
                'model_template': model_templates[i],
                'model_type': model_types[i],
                'morphology': morphologies[i],
                'mtype': mtypes[i],
            }
        )
        session.commit()

### Astrocytes

In [ ]:
morphology_folder = components['morphologies_dir']
print(morphology_folder)

morphology_files = glob.glob(os.path.join(morphology_folder, '*.h5'))
for morphology_file in morphology_files:
    m = morphio.Morphology(morphology_file)
    break

In [ ]:
m = morphio.Morphology(morphology_files[0])

In [ ]:
section_types = dict()
section_types[morphio.SectionType.undefined] = 0
section_types[morphio.SectionType.soma] = 1
section_types[morphio.SectionType.axon] = 2
section_types[morphio.SectionType.basal_dendrite] = 3
section_types[morphio.SectionType.apical_dendrite] = 4

In [ ]:
with Session(engine) as session:
    session.execute('DELETE FROM %s.section' % db_schema)
    session.commit()

with Session(engine) as session:
    for morphology_guid in tqdm(range(len(morphologies))):
        morphology=morphologies[morphology_guid]

        morphology_file = os.path.join(morphology_folder, morphology + '.h5')
        m = morphio.Morphology(morphology_file)

        for section in m.sections:
            section_type_guid = section_types[section.type]
            parent = -1
            if not section.is_root:
                parent =  section.parent.id

            points = list()
            for i in range(len(section.points)):
                point = section.points[i]
                points.append(point[0])
                points.append(point[1])
                points.append(point[2])
                points.append(section.diameters[i])
            buffer = np.array(points, dtype=np.float32).tobytes()

            session.execute(
                'INSERT INTO %s.section VALUES (:morphology_guid, :section_guid, :section_parent_guid, :section_type_guid, :points)' % db_schema,
                {
                    'morphology_guid': morphology_guid,
                    'section_guid': section.id,
                    'section_parent_guid': parent,
                    'section_type_guid': section_type_guid,
                    'points': buffer
                }
            )
        if morphology_guid % 10 == 0:
            session.commit()
    session.commit()

### Micro-domains

In [ ]:
microdomains_file = json['networks']['nodes'][2]['populations']['astrocytes']['microdomains_file']
microdomains_file = os.path.join(data_folder, 'build', microdomains_file)
microdomains = h5py.File(microdomains_file)

data = microdomains['data']
offsets = microdomains['offsets']
points = data['points']
triangle_indices = data['triangle_data']

offset_points = offsets['points']
offset_triangle_indices = offsets['triangle_data']

astrocyte_guid = 0
nb_astrocytes = len(offset_points) 

with Session(engine) as session:
    
    session.execute('DELETE FROM %s.micro_domain' % db_schema)
    
    for i in tqdm(range(nb_astrocytes - 1)):
        points_as_floats = list()
        triangle_indices_as_ints = list()

        point_offset = offset_points[i]
        nb_points = offset_points[i + 1] - point_offset
        for j in range(point_offset, point_offset + nb_points):
            points_as_floats.append(float(points[j][0]))
            points_as_floats.append(float(points[j][1]))
            points_as_floats.append(float(points[j][2]))

        triangle_offset = offset_triangle_indices[i]
        nb_triangles = offset_triangle_indices[i + 1] - triangle_offset
        for j in range(triangle_offset, triangle_offset + nb_triangles):
            # What is the first value of the triangle indices?
            triangle_indices_as_ints.append(triangle_indices[j][1])
            triangle_indices_as_ints.append(triangle_indices[j][2])
            triangle_indices_as_ints.append(triangle_indices[j][3])

        vertices_buffer = np.array(points_as_floats, dtype=np.float32).tobytes()
        indices_buffer = np.array(triangle_indices_as_ints, dtype=np.uint32).tobytes()
    
        session.execute(
            'INSERT INTO %s.micro_domain VALUES (:guid, :vertices, :indices)' % db_schema,
            {
                'guid': astrocyte_guid,
                'vertices': vertices_buffer,
                'indices': indices_buffer,
            }
        )
        session.commit()
        astrocyte_guid += 1